In [2]:
!pip install transformers

### imports

In [4]:
import json
from transformers import pipeline
from transformers import BertModel, BertTokenizer, TFBertForSequenceClassification, TFAutoModelForSequenceClassification
import numpy as np
from sklearn.model_selection import train_test_split
from pprint import pprint
from tensorflow.keras.optimizers import Adam

### Use transformers get a model for predicting MGT

In [4]:
pipe = pipeline(model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### load data from jsonl

In [5]:
def load_data(file_path):
  data_list = []
  with open(file_path, 'r') as file:
    for line in file:
        # load each line as a JSON object
        data = json.loads(line.strip())
        data_list.append(data)
  return data_list


In [6]:
file_path = 'drive/MyDrive/models/subtaskA_dev_monolingual.jsonl'

In [7]:
data_loaded = load_data(file_path)

### get a random sample from data

In [5]:
test_data = data_loaded[100]['text']
print(test_data)
print("label is: ", data_loaded[100]['label'])

If you can't get your Samsung Galaxy S3 connected to your computer using its default connection method -- MTP or PTP -- you'll need to use an alternative way.  This guide will show you how to do that by enabling ADB debugging mode instead.   Note that this process may void... If you're still having trouble getting your S3 to work properly after following these steps, try contacting Samsung directly via their support page here. You can also visit our troubleshooting section if you'd like more help fixing other issues related to your Android smartphone. Restart both devices before continuing so they are ready to be reconnected.    The first thing we recommend doing is trying out another USB cord and/or USB port since many users have reported success simply changing cables/ports while attempting to fix problems between phones/computers.     To check which ports are currently being used,...   -  Plug one end of the new USB cord into your computer's charging port..
-  Plug the other end of 

### Test using pipeline

In [22]:
# test this using the pipeline
result = pipe(test_data,
              candidate_labels=["Written by humans", "Written by AI"],)

In [49]:
print(type(test_data))

<class 'str'>


In [31]:
pprint(result)

{'labels': ['Written by humans', 'Written by AI'],
 'scores': [0.5420032143592834, 0.45799681544303894],
 'sequence': "If you can't get your Samsung Galaxy S3 connected to your "
             'computer using its default connection method -- MTP or PTP -- '
             "you'll need to use an alternative way.  This guide will show you "
             'how to do that by enabling ADB debugging mode instead.   Note '
             "that this process may void... If you're still having trouble "
             'getting your S3 to work properly after following these steps, '
             'try contacting Samsung directly via their support page here. You '
             "can also visit our troubleshooting section if you'd like more "
             'help fixing other issues related to your Android smartphone. '
             'Restart both devices before continuing so they are ready to be '
             'reconnected.    The first thing we recommend doing is trying out '
             'another USB cord an

In [8]:
def prepare_test_data():
  texts = []
  labels = []
  for data in data_loaded:
    texts.append(data['text'])
    labels.append(data['label'])
  return texts,labels


In [9]:
x_test, y_test = prepare_test_data()

In [10]:
print(len(x_test))

5000


In [11]:
x_test_sample = x_test[:3]
y_test_sample = y_test[:3]

In [55]:
results = pipe(x_test_sample,
              candidate_labels=["Written by humans", "Written by AI"],)

In [ ]:
pprint(results)

In [56]:
# verify results
correct = 0
for i in range(len(results)):
    # take the index of the label with the highet score from scores
    if results[i]['scores'].index(max(results[i]['scores'])) == y_test[i]:
      correct +=1
print("Accuracy is : ", correct / len(results))

Accuracy is :  0.0


# Zero-shot classification


In [64]:
classifier = pipeline("zero-shot-classification")
classifier(
    x_test_sample,
    candidate_labels=["human", "AI"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'sequence': "Giving gifts should always be enjoyable.  However, it may become stressful when trying to find that perfect present.   This wikiHow will help you figure out exactly what you'd love to receive this year!   If you're having trouble deciding between two different items (or more), try making lists of advantages/disadvantages so you'll know which one would make the best choice.    Make sure it's appropriate - some people don't appreciate receiving certain types of presents from their friends and/or family members.     Don't forget to include any special requests you've already made before now; these might have been forgotten by others who haven't seen them yet.        Write down all... Continue reading →\n\nIf you can't think of anything specific right away but still feel like getting something nice,  consider giving yourself a treat instead!  Here are just a few suggestions:  Buying yourself flowers Gifting yourself tickets to see your favorite band Take time off work Treat 

# Fine-tuning: using BERT pre-trained model!

In [8]:
x_test, y_test = prepare_test_data()

In [13]:
print(len(x_test))

5000


### Split data into train and test

In [10]:
# split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.2, random_state=42)

In [11]:
# print the shapes of the train and test set
print(len(X_train), len(y_train), len(X_test), len(y_test))

4000 4000 1000 1000


In [10]:
# load the pre-trained BERt tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# convert labels to numpy array
# the labels are already a list of 0 and 1s, so we can just convert that directly to a NumPy array without tokenization!
y_train = np.array(y_train)
y_test = np.array(y_test)

In [18]:
# Tokenize and pad the input sequences
X_train_tokens = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
X_test_tokens = tokenizer(X_test, padding=True, truncation=True, max_length=128, return_tensors="tf")


In [ ]:
print(X_train_tokens[0])

In [80]:
# prepare the model for training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_tokens['input_ids'], y_train, epochs=2, batch_size=32, validation_data=(X_test_tokens['input_ids'], y_test))

Epoch 1/2
  3/125 [..............................] - ETA: 1:45:18 - loss: 5.4143 - accuracy: 0.4688

In [ ]:
# evaluate the model on the test set:
scores = model.evaluate(X_test_tokens['input_ids'], y_test)
print("Accuracy:", scores[1])

### version 2

In [14]:
from transformers import AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [14]:
data_loaded[0]['text']

"Giving gifts should always be enjoyable.  However, it may become stressful when trying to find that perfect present.   This wikiHow will help you figure out exactly what you'd love to receive this year!   If you're having trouble deciding between two different items (or more), try making lists of advantages/disadvantages so you'll know which one would make the best choice.    Make sure it's appropriate - some people don't appreciate receiving certain types of presents from their friends and/or family members.     Don't forget to include any special requests you've already made before now; these might have been forgotten by others who haven't seen them yet.        Write down all... Continue reading →\n\nIf you can't think of anything specific right away but still feel like getting something nice,  consider giving yourself a treat instead!  Here are just a few suggestions:  Buying yourself flowers Gifting yourself tickets to see your favorite band Take time off work Treat yourself to lu

In [16]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(X_train, return_tensors="np", padding=True, truncation=True, max_length=128)

In [17]:
tokenized_data = dict(tokenized_data)

In [20]:
from transformers import AutoModelForSequenceClassification

In [23]:
# Load and compile our model
# for subtask A
id2label = {0 : "human" , 1: "machine"}
label2id = {"human": 0, "machine": 1}
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.compile(Adam(3e-5))  # No loss argument!
# Hugging Face models automatically choose a loss that is appropriate for their task

In [ ]:
model.fit(tokenized_data, y_train)

 13/125 [==>...........................] - ETA: 1:45:07 - loss: 0.7109

## Version 3

In [ ]:
!pip install Datasets

In [71]:
import pandas as pd
from datasets import Dataset
from transformers import DataCollatorWithPadding
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [94]:
def preprocess_function(examples, tokenizer, max_length=128):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=max_length, return_tensors="tf")

In [95]:
def get_data(train_path):
   train_df = pd.read_json(train_path, lines=True)
   train_df, test_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)
   return train_df, test_df


In [96]:
train_df, test_df = get_data(file_path)

In [97]:
train_df["text"]

2413    They have been accused of being greedy and man...
317     Minnie Mouse is an iconic character from Disne...
1067    Paul McCartney has been touring since the 1960...
3637      We present a detailed analysis of XMM-Newton...
3964      Opacities and chemical abundance data are cr...
                              ...                        
4740    We have updated the pdf of the paper after the...
291     This is an excellent recipe because it's very ...
80      Photoshop is available as both a free web-base...
631      Flick up a switch to the back of the presser ...
2118    Bernie Sanders is an American politician who h...
Name: text, Length: 4000, dtype: object

In [98]:
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1 }

In [99]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [100]:
 # pandas dataframe to huggingface Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [101]:
tokenized_train_dataset = train_dataset.map(lambda example: preprocess_function(example, tokenizer, max_length=128))
tokenized_test_dataset = test_dataset.map(lambda example: preprocess_function(example, tokenizer, max_length=128))

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [102]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [103]:
training_args = {
        'learning_rate': 2e-5,
        'num_train_epochs': 3,
        'per_device_train_batch_size': 16
    }



In [104]:
model.compile(optimizer=Adam(learning_rate=training_args['learning_rate']),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[SparseCategoricalAccuracy('accuracy')])


In [105]:
X_train = np.vstack(tokenized_train_dataset["input_ids"])
y_train = np.array(train_df["label"])

In [106]:
history = model.fit(X_train,
                    epochs=training_args['num_train_epochs'],
                    batch_size=training_args['per_device_train_batch_size']
                    )

Epoch 1/3


AttributeError: ignored